In [ ]:
import numpy as np
from scipy.optimize import root
from functools import partial


# Advanced Applied Econometrics: static discrete choice with market-level data.

# Problem Set: Preliminaries

The aim of this problem set is to refresh (or make you familiar with) some basic concepts -- notably GMM estimation, numerically solving systems of nonlinear equations, and the intuition for a contraction mapping - that will be essential in subsequent sessions and problem sets (and in large parts of the literature).

Berry, Steven T. (1994), "Estimating Discrete Choice Models of Product Differentiation," *Rand Journal of Economics*, 25 (2), 242-262.

## Problems


### 1. A simple demand estimation example.

We assume consumer $i$ chooses one unit of product $j\in J$ or an outside good (e.g. no purchase) to obtain utility 

\begin{equation}
u_{ijt} = x_{jt}\beta + \alpha p_{jt} + \xi_{jt} + \varepsilon_{ijt} = \delta_{jt} + \varepsilon_{ijt}
\tag{1}
\end{equation} 


where $(x_{jt},p_{jt})$ are observable characteristics and price, $\xi_{jt}$ is an unobservable characteristic, and an idiosyncratic error term $\varepsilon_{ijt}$ assumed i.i.d. extreme value type 1. The utility of the outside good is normalized such that $\delta_{0t}=0$. The assumption of utility-maximizing consumers and the distribution of $\varepsilon_{ijt}$ yields the logit choice probabilities:


\begin{equation}
s_{jt}(\delta_t) = \frac{\exp(\delta_{jt})}{1+\sum_{l=1}^J \exp(\delta_{lt})}.
\tag{2}
\end{equation}



##### a) Simulate product-level data based on the described model assuming the following:

- $J=10$ products are sold in $T=25$ markets (size $L_t=1$) by single-product firms.
- Two observable product characteristics $x_{jt}=(1,x_{jt}^1)$, with $x_{jt}^1 \sim U(1,2)$.
- Marginal cost $c_{jt}=x_{jt}\gamma_1 + w_{jt}\gamma_2 + \omega_{jt}$.
- Three observable cost shifters $w_{jt}=(w_{jt}^1,w_{jt}^2,w_{jt}^3)$, all i.i.d. $U(0,1)$.
- Marginal cost parameters: $\gamma_1=(0.7,0.7)$ and $\gamma_2=(1,1,1)$.
- Unobserved demand and cost characteristic $(\xi_{jt},\omega_{jt}) \sim N(0,\sigma_c)$ with $\sigma_c = \left[
\begin{array}{cc}
1 & 0.7 \\
0.7 & 1 \\
\end{array}\right]$.
- Assume perfect competition on price so that $p_{jt}=c_{jt}$.
- Preference parameters $\beta=(2,2), \alpha=-2$.

#### Set seed for reproducibility:

In [ ]:
np.random.seed(100)

#### Set market structure

In [ ]:
# number of markets, T
num_markets = 25
# number of brands per market, J
num_brands = 10

num_obs = num_markets * num_brands

#### Create observed characteristic and shifters

In [ ]:
# Product Attribute uniformly distributed, i.e. U(1,2)
random_obs_product_chars = 1 + np.random.rand(num_obs)
# Product attribute constant
constant_obs_product_chars = np.ones(num_obs)
# Joint both
obs_product_characteristics = np.column_stack(
    [constant_obs_product_chars, random_obs_product_chars]
).reshape((num_markets, num_brands, 2))

# Cost Shifters, U(0,1)
cost_shifters = np.random.rand(num_markets, num_brands, 3)

#### Create unobserved characteristics

In [ ]:
# Degree of endogeneity
# covariance in share and price eq. errors
true_covariance = np.array([[1, 0.7], [0.7, 1]])
true_mean = np.zeros(2)

# Unobserved characteristics: omeg and Ksi  ## #Do we need the reshape?
unobs_characteristics = np.random.multivariate_normal(
    mean=true_mean, cov=true_covariance, size=(num_markets, num_brands)
)
unobs_demand_characteristic = unobs_characteristics[:, :, 0]
unobs_cost_characteristic = unobs_characteristics[:, :, 1]

#### Assume true parameters

In [ ]:
# Marginal cost Equation Parameters
obs_cost_parameters = np.array([0.7, 0.7])
unobs_cost_parameters = np.array([1, 1, 1])

# Utility parameters
characteristic_preference_params = np.array([2, 2])
price_preference_param = -2

#### Calculate prices, mean utility and shares

To simulate the data, you must code a function computing a $(T\times J)\times 1$ vector containing market shares using equation (2).

In [ ]:
def calc_marginal_costs(gamma_1, gamma_2, x, w, omega):
    """This function calculates the marginal costs given product characteristics and cost shifters.
    Parameters
    ----------
    gamma_1 (np.ndarray):
        Marginal cost parameters for product characteristics. Shape is (num_characteristics,).
    gamma_2 (np.ndarray):
        Marginal cost parameters for cost shifters. Shape is (num_cost_shifters,).
    x (np.ndarray):
        Product characteristics. Shape is (num_markets, num_products, num_characteristics).
    w (np.ndarray):
        Cost shifters. Shape is (num_markets, num_products, num_cost_shifters).

    Returns
    -------
    np.ndarray
        Marginal costs. Shape is (num_markets, num_products).
    """
    return x @ gamma_1 + w @ gamma_2 + omega

In [ ]:
def calc_shares_from_mean_utility(delta):
    """This function calculates the market shares given mean utility.
    Parameters
    ----------
    delta (np.ndarray):
        Mean utility. Shape is (num_markets, num_products).

    Returns
    -------
    np.ndarray
        Market shares. Shape is (num_markets, num_products).
    """
    delta_exp = np.exp(delta)
    shares = np.divide(delta_exp.T, 1 + delta_exp.sum(axis=1)).T
    return shares

In [ ]:
def calc_shares_and_mean_utility(x, beta, p, alpha, ksi):
    """This function calculates the market shares and mean utility
    given product characteristics, prices and unobserved demand characteristics.

    Parameters
    ----------
    x (np.ndarray):
        Product characteristics. Shape is (num_markets, num_products, 2).
    beta (np.ndarray):
        Preference parameters for product characteristics. Shape is (2,).
    p (np.ndarray):
        Prices. Shape is (num_markets, num_products).
    alpha (float):
        Preference parameter for price.
    ksi (np.ndarray):
        Unobserved demand characteristics. Shape is (num_markets, num_products).
    """
    delta = x @ beta + p * alpha + ksi
    shares = calc_shares_from_mean_utility(delta)
    return shares, delta

In [ ]:
marginal_costs = prices_perfect_comp = calc_marginal_costs(
    gamma_1=obs_cost_parameters,
    gamma_2=unobs_cost_parameters,
    x=obs_product_characteristics,
    w=cost_shifters,
    omega=unobs_cost_characteristic,
)

In [ ]:
shares_perfect_comp, mean_utility_perfect_comb = calc_shares_and_mean_utility(
    x=obs_product_characteristics,
    beta=characteristic_preference_params,
    p=prices_perfect_comp,
    alpha=price_preference_param,
    ksi=unobs_demand_characteristic,
)

##### b) The simulated data at hand, 

- Forget parameter values and, following Berry (1994), estimate $\{\alpha,\beta\}$ using OLS and report the results.

In [ ]:
def ols_formula(y, x):
    """This function calculates the OLS estimates and standard errors.

    Parameters
    ----------
    y (np.ndarray):
        Dependent variable. Shape is (num_markets * num_products).
    x (np.ndarray):
        Independent variables. Shape is (num_markets * num_products, num_independent_variables).

    Returns
    -------
    coeffs (np.ndarray):
        OLS estimates. Shape is (num_independent_variables,).
    std_errors (np.ndarray):
        Standard errors. Shape is (num_independent_variables,).
    """
    inverse_covars = np.linalg.inv(x.T @ x)
    # OLS estimator formular
    coeffs = inverse_covars @ (x.T @ y)

    # Now estimation of standard errors
    projection = x @ coeffs

    residuals = y - projection
    squared_sum_residuals = residuals @ residuals

    degrees_of_freedom = x.shape[0] - x.shape[1]
    covariance_est = (squared_sum_residuals / degrees_of_freedom) * inverse_covars
    std_errors = np.sqrt(np.diag(covariance_est))
    return coeffs, std_errors

In [ ]:
# Generate demand objects for OLS:

# Log share of the outside good
log_outside_shares = np.log(1 - shares_perfect_comp.sum(axis=1))
# Log share difference
log_share_diff = np.log(shares_perfect_comp) - log_outside_shares[:, np.newaxis]
# Reshape to get y vector
y_ols = log_share_diff.reshape((num_markets * num_brands, ))

# Generate and fill up covariates vector. Three entries.
x_ols = np.empty((num_obs, 3))
x_ols[:, :2] = obs_product_characteristics.reshape(num_obs, 2)
x_ols[:, 2] = prices_perfect_comp.flatten()

ols_formula(y=y_ols, x=x_ols)

- Estimate $\{\alpha,\beta\}$ by GMM using as instrumental variables the observable characteristics $x_{jt}$ and cost shifters $w_{jt}$, and report the results. 

First, write the GMM estimation function:

In [ ]:
def two_sls_formula(y, x, z):
    """This function calculates the 2SLS estimates and standard errors.
    Parameters
    ----------
    y (np.ndarray):
        Dependent variable. Shape is (num_markets * num_products).
    x (np.ndarray):
        Independent variables. Shape is (num_markets * num_products, num_independent_variables).
    z (np.ndarray):
        Instrumental variables. Shape is (num_markets * num_products, num_instrumental_variables).

    Returns
    -------
    coeffs (np.ndarray):
        2SLS estimates. (Shape is (num_independent_variables,))
    std_errors (np.ndarray):
        Standard errors. (Shape is (num_independent_variables,))
    """
    # Step 1: 2SLS (homoscedastic errors), "weighting matrix" W=inv(Z'Z)
    degrees_of_freedom = x.shape[0] - x.shape[1]

    norm = np.mean(np.mean(z.T @ z))
    W = np.linalg.inv((z.T @ z) / norm) / norm

    mid_matrix = z @ W @ z.T
    coeffs = np.linalg.inv(x.T @ mid_matrix @ x) @ (x.T @ mid_matrix @ y)

    residuals = y - x @ coeffs
    squared_sum_residuals = residuals.T @ residuals

    inverse_covariates = np.linalg.inv(x.T @ mid_matrix @ x)
    estimator_cov = (squared_sum_residuals / degrees_of_freedom) * inverse_covariates # check this line: should divide element-wise and multiply by diagonal of inverse_covariates
    std_errors = np.sqrt(np.diag(estimator_cov))

    return coeffs, std_errors, residuals


def optimal_gmm(y, x, z, residuals_raw):
    """This function calculates the optimal GMM estimates and standard errors.
    Parameters
    ----------
    y (np.ndarray):
        Dependent variable. Shape is (num_markets * num_products).
    x (np.ndarray):
        Independent variables. Shape is (num_markets * num_products, num_independent_variables).
    z (np.ndarray):
        Instrumental variables. Shape is (num_markets * num_products, num_instrumental_variables).
    residuals_raw (np.ndarray):
        Residuals from the first step of 2SLS. Shape is (num_markets * num_products).

    Returns
    -------
    coeffs (np.ndarray):
        Optimal GMM estimates. Shape is (num_independent_variables,).
    std_errors (np.ndarray):
        Standard errors. Shape is (num_independent_variables,).
    """
    degrees_of_freedom = x.shape[0] - x.shape[1]

    instrument_residuals = z * residuals_raw[:, np.newaxis]
    updated_weight_mat = instrument_residuals.T @ instrument_residuals

    mid_matrix = z @ np.linalg.inv(updated_weight_mat) @ z.T
    coeffs = np.linalg.inv(x.T @ mid_matrix @ x) @ (x.T @ mid_matrix @ y)

    residuals = y - x @ coeffs
    squared_sum_residuals = residuals.T @ residuals

    inverse_covariates = np.linalg.inv(x.T @ mid_matrix @ x)
    covariance_estimated = (
        squared_sum_residuals / degrees_of_freedom
    ) * inverse_covariates
    std_errors = np.sqrt(np.diag(covariance_estimated))
    return coeffs, std_errors

Estimate the model.

In [ ]:
# Generate array for instrument 1
instruments_1 = np.empty((num_obs, 3))
instruments_1[:, :2] = obs_product_characteristics.reshape(num_obs, 2)

sum_product_characteristics = obs_product_characteristics[:, :, 1].sum(axis=1)[
    :, np.newaxis
]
instruments_1[:, 2] = (
    sum_product_characteristics - obs_product_characteristics[:, :, 1]
).flatten()

In [ ]:
# By 2SLS
estimates_2SLS, stdt_2SLS, residuals_2SLS = two_sls_formula(
    y=y_ols, x=x_ols, z=instruments_1
)
estimates_2SLS, stdt_2SLS

In [ ]:
# And by optimal GMM
estimates_opt_GMM, stdt_opt_GMM = optimal_gmm(
    y=y_ols, x=x_ols, z=instruments_1, residuals_raw=residuals_2SLS
)
estimates_opt_GMM, stdt_opt_GMM

In [ ]:
# Create instruments 2 and estimate by 2SLS
instruments_2 = np.concatenate((obs_product_characteristics.reshape(num_obs, 2), cost_shifters.reshape(num_obs, 3)), axis=1)
estimates_2SLS, stdt_2SLS, residuals_2SLS = two_sls_formula(
    y=y_ols, x=x_ols, z=instruments_2
)
estimates_2SLS, stdt_2SLS

In [ ]:
# And by optimal GMM
optimal_gmm(y=y_ols, x=x_ols, z=instruments_2, residuals_raw=residuals_2SLS)

### 2. Solving for a static industry equilibrium. 

Now consider an imperfectly competitive industry, that is $p_{jt} \neq c_{jt}$. Take the market share equation, cost and demand parameters from above as given. Assume that single-product firms j maximize profits given by

\begin{equation}
\pi_{jt} = (p_{jt} - c_{jt})s_{jt}L_{t},
\end{equation}

where $c_{jt}$ are marginal cost and $L_{t}$ market size, so that the system of FOC for a Nash equilibrium:
    
\begin{equation}
s_{jt} + (p_{jt} - c_{jt})\frac{\partial s_{jt}}{\partial p_{jt}} = 0
\end{equation}

With multi-product firms, it is useful to write expression (4) in vector notation as $s_t+\Delta_t(p_t-c_t)=0,$ where $\Delta_t(j,k)$ denotes a diagonal matrix of own-price derivatives and off-diagonal elements according to market structure. With single-product firms, off-diagonal elements are equal to zero so that $\Delta_t$ can be collapsed to a vector. If marginal cost are known, we obtain the supply side by solving the system for $c_t$:

\begin{equation}
p_t+\Delta_t^{-1}s_t = c_t
\end{equation}

For single-product firms, $\partial s_{jt} / \partial p_{jt}$ is given by $\alpha s_{jt} (1-s_{jt})$.


##### a) Generate a $(T\times J)\times 1$ vector $\Delta$ containing market share derivatives with respect to own-price.

In [ ]:
def market_derivatives_one_product_firm(shares, alpha):
    """This function calculates the market share derivatives for a single product firm.

    Parameters
    ----------
    shares (np.ndarray):
        Market shares. Shape is (num_markets * num_products,).
    alpha (float):
        Price preference parameter.

    Returns
    -------
    derivatives (np.ndarray):
        Market share derivatives. Shape is (num_markets * num_products,).
    """
    return alpha * shares * (1 - shares)

In [ ]:
derivatives = market_derivatives_one_product_firm(
    shares=shares_perfect_comp, alpha=price_preference_param
)
derivatives.shape

##### b) compute a $(T\times J)\times 1$ vector $p$ containing (Bertrand-Nash) equilibrium prices, using equation (5) and a root-finding function, and report the mean, minimum, and maximum industry price.

In [ ]:
def root_diff(p, mc, x, beta, alpha, ksi):
    """This is the auxiliary function for the root finding. The prices p will be given
    as a flat vector and the function needs to return a flat vector.

    Parameters
    ----------
    p (np.ndarray):
        Prices. Shape is (num_markets * num_products,).
    mc (np.ndarray):
        Marginal costs. Shape is (num_markets * num_products,).
    x (np.ndarray):
        Product characteristics. Shape is (num_markets * num_products, num_characteristics).

    Returns
    -------
    diff (np.ndarray):
        Difference between left and right hand side of the root finding problem. Shape is (num_markets * num_products,).
    """
    p_int = p.reshape((x.shape[0], x.shape[1]))
    shares, _ = calc_shares_and_mean_utility(x, beta, p_int, alpha, ksi)
    market_share_dev = market_derivatives_one_product_firm(shares, alpha)
    return (mc - p_int - np.divide(shares, market_share_dev)).flatten()

In [ ]:
diff_from_price = partial(
    root_diff,
    mc=marginal_costs,
    x=obs_product_characteristics,
    beta=obs_cost_parameters,
    alpha=price_preference_param,
    ksi=unobs_demand_characteristic,
)

In [ ]:
result_root = root(diff_from_price, prices_perfect_comp)
prices_imperfect_comp = result_root["x"].reshape((num_markets, num_brands))
print([np.min(prices_imperfect_comp),np.mean(prices_imperfect_comp),np.max(prices_imperfect_comp)])

print(f'{result_root.success = }')
print(result_root.message)

### 3. Contraction Mapping.

Newton's method is an important tool in nonlinear optimization. It is used to find roots of a function $f(x)$. Write a function finding the number $\sqrt{a}$ by iterating $x_{t+1}=x_t-\frac{f(x_t)}{f'(x_t)}$. Hint: $\sqrt{a}$ is the positive root of $f(x)=a-x^2$.

Note: by the Contraction Mapping Principle (Banach Fixed Point Theorem) this method is a contraction under the conditions that 
1) $f(x)$ has a continuous second derivative, 
2) $f'(x) \neq 0\ \forall\ x \in \mathbb{R}$, and 
3) a $q \in (0,1)$ exists such that $|f(x)f''(x)| \leq q |f'(x)|^2\ \forall\ x \in \mathbb{R}$.


In [ ]:
# Generate contraction mapping
def newton_sqrt(a, tol=1e-12):
    """This function calculates the square root of a number a using Newton's method.

    Parameters
    ----------
    a (float):
        Number to calculate the square root of.
    tol (float):
        Tolerance level for the convergence of the algorithm.

    Returns
    -------
    x_t1 (float):
        Square root of a.
    """
    if a < 0:
        raise ValueError("Number a should be positive.")

    x_t = a / 2
    x_t1 = a / 2 - 1
    i = 0

    while abs(x_t1 - x_t) > tol:
        x_t = x_t1
        f_t = a - x_t**2
        f_t_derivative = -2 * x_t
        x_t1 = x_t - f_t / f_t_derivative
        i += 1
        print(f"Value at iteration {i} is {x_t1:.10f}.")

    return x_t1

In [ ]:
newton_sqrt(16)